In [ ]:
!mc cp s3/$VAULT_TOP_DIR/Accords/Construction_dataset_public/Dataset_public_accords_teletravail_Dares.parquet .

In [ ]:
!mc cp -r s3/$VAULT_TOP_DIR/Accords/chroma_db .

In [ ]:
import pandas as pd
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain_chroma import Chroma
from langchain.embeddings import OpenAIEmbeddings, OllamaEmbeddings
from langchain.document_loaders import TextLoader
from langchain.schema import Document, Generation, LLMResult
from langchain.llms import Ollama, BaseLLM
from langchain.chains import StuffDocumentsChain, RetrievalQA, LLMChain
from langchain_core.prompts import PromptTemplate
from langchain_community.llms import OpenAI
from langchain_text_splitters import CharacterTextSplitter
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from pathlib import Path
import json
import requests

#"seed":1337,"temperature":1,"num_ctx":100,
#,"options":{"stop":[",","."]}
class LocalOllamaLLM(BaseLLM):
    api_url : str
    def _generate(self, prompt, stop):
        response = requests.post(f"{self.api_url}/api/generate", json={"model": "llama3.1", "prompt": str(prompt) })
        response.raise_for_status()
        response_text=''.join([json.loads(line)['response'] for line in response.text.splitlines()])
        generations=[]
        generations.append([Generation(text=response_text)])
        return LLMResult(generations=generations)


    def _llm_type(self):
        return "local"  



def search_and_invoke_llm(vector_store,index,query,k=5):
    if k==0:
        print(f"bug with {index}")
        return None
    elif k==1:
        pass
        #print(f" trying k=1 : {index}")
    else:
        pass
    try:
        retriever=vector_store.as_retriever(
        search_kwargs={
                "k": k, 
                "filter": {'index': index}
            }
        )
        chain = create_retrieval_chain(retriever, question_answer_chain)
        result=chain.invoke({"input": query})
        return result
    except:
        search_and_invoke_llm(vector_store,index,query,k=k-1)
    return None



llm = LocalOllamaLLM(api_url="http://127.0.0.1:11434")

embedder = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")

system_prompt = (
    " Répondez à la question posée "
    " Utilisez le contexte (sélection des meilleurs paragraphes liés à la question) donné pour répondre à la question "
    " Répondez en bullet point comme ceci : "
    " ### Règles : "
    " * (règle 1) "
    " * (règle 2) "

    #" Si la réponse ne se trouve pas dans le contexte, répondez par 'Non'"
    " Contexte : {context}  "
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(llm, prompt)

In [ ]:
file="Dataset_public_accords_teletravail_Dares.parquet"
df=pd.read_parquet(file)

In [ ]:
vector_store = Chroma(embedding_function=embedder,persist_directory="./chroma_db")

In [ ]:
Q0="Est-ce que les cadres télétravaillent différemment des non-cadres ?"

LIST_OF_QUESTIONS=[Q0]

In [ ]:
from glob import glob
already_done={el.split("/")[1].split(".")[0] for el in glob("results/*.answer")}
new_dir = Path('results').mkdir(exist_ok=True)
for index, row in df.iterrows():
    answer=""
    if index not in already_done:

        for Q in LIST_OF_QUESTIONS:
            if ans:=search_and_invoke_llm(vector_store,index,Q,k=2):
                answer_txt=ans['answer']
                answer += answer_txt
            answer += "\n-----\n"
        print(index,df.nombre_jour_hebdo_TT_annotee[index], answer)
        if answer:
            with open(f"results/{index}.answer","w") as f:
                f.write(answer)

In [ ]:
!mc cp -r results s3/$VAULT_TOP_DIR/Accords/Teletravail/Runs/Llama3.1/Cadres/